## Рекомендательные системы
### Урок 3. Коллаборативная фильтрация

#### 1) Попытаться ответить на вопросы/выдвинуть гипотезы

Какие вопросы?

#### 2) Доделать прошлые домашния задания
Ок

#### 3) Прочитать статьи BM25/MatrixFactorization
Ок

#### 4) Поэкспериментировать с ALS (grid-search)

In [4]:
import pandas as pd
import numpy as np
import collections
import time

from scipy.sparse import csr_matrix
from scipy import stats
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import warnings
warnings.filterwarnings('ignore')

In [7]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def get_recommendations(user, model, sparse_user_item, N=5):
    return [id_to_itemid[rec[0]] for rec in 
            model.recommend(userid=userid_to_id[user], 
                            user_items=sparse_user_item,
                            N=N, 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)]


def CheckWeight(model, user_item_matrix, valid):
    sparse_user_item = csr_matrix(user_item_matrix).tocsr()
    model.fit(csr_matrix(user_item_matrix).T, show_progress=False)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    valid['als'] = valid['user_id'].apply(lambda x: get_recommendations(x, model=model, sparse_user_item=sparse_user_item, N=5))
    return valid.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()


data = pd.read_csv('./retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

data['norm_retail_disc'] =  (data['retail_disc']-data['retail_disc'].min())/(data['retail_disc'].max() - data['retail_disc'].min())
data['week_no_52'] =  data['week_no'] % 52
data['week_no_52'] =  (data['week_no_52']-data['week_no_52'].min())/(data['week_no_52'].max() - data['week_no_52'].min())

data['sec_trans_time'] = (60*data['trans_time'].astype(int)/100 + data['trans_time'] % 100)*60
data['sec_trans_time'] =  (data['sec_trans_time']-data['sec_trans_time'].min())/(data['sec_trans_time'].max() - data['week_no_52'].min())

test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

item_features = pd.read_csv('./product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

result_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
result_train.columns=['user_id', 'actual']
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix_quantity = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', aggfunc='count', 
                                  fill_value=0).astype(float)

user_item_matrix_norm_retail_disc = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='norm_retail_disc', aggfunc='mean', 
                                  fill_value=0).astype(float)

user_item_matrix_week_no_52 = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='week_no_52', aggfunc='mean', 
                                  fill_value=0).astype(float)

user_item_matrix_sec_trans_time = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sec_trans_time', aggfunc='mean', 
                                  fill_value=0).astype(float)


userids = user_item_matrix_quantity.index.values
itemids = user_item_matrix_quantity.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

model = AlternatingLeastSquares(factors=100, #k
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True,
                                use_gpu=False, 
                                num_threads=0)

lst_user_item_matrix = [user_item_matrix_quantity, user_item_matrix_norm_retail_disc, 
user_item_matrix_week_no_52, user_item_matrix_sec_trans_time]

res = []

for uim in lst_user_item_matrix:
    res.append(CheckWeight(model, uim, result))

columns=['quantity', 'retail_disc', 'week_no_52', 'sec_trans_time']
for c,r in zip(columns, res):
    print(c,'\t', r)

ValueError: user_items must contain 1 row for every user in userids

In [ ]:
BestModel = collections.namedtuple('BestModel', ['model', 'score_valid', 'score_test', 'param','time'])
params = {
    'factors' : [25, 50, 75, 100, 300],
    'regularization' : [1, 0.1, 0.01, 0.05, 0.001],
    'iterations': 15,
    'calculate_training_loss': True,
    'use_gpu': False,
    'num_threads': 0
}
best_model = BestModel(model=None, score_valid=0, score_test=0, param=None, time=0)

In [ ]:
models=[]
for param in params:
    start_time = time.time()
    model = AlternatingLeastSquares(**param)
    score_valid = CheckWeight(model,user_item_matrix_norm_retail_disc,result)
    score_test = CheckWeight(model,user_item_matrix_norm_retail_disc,result_train)
    tm = (time.time() - start_time)
    models.append(BestModel(model, score_valid, score_test,
                            params, tm))
    if best_model.score_valid < score_valid:
        best_model = models[-1]

In [ ]:
best_model.param

In [ ]:
best_model.score_valid